# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute

from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
import os
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws)
experiment_name = 'labor-prediction-experiment'

experiment=Experiment(ws, experiment_name)




Workspace.create(name='quick-starts-ws-133192', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-133192')


In [4]:
datastore_name='workspaceblobstore'
datastore=Datastore.get(ws,datastore_name)

datastore_path = [(datastore, 'UI/01-03-2021_062128_UTC/LaborPredictionData6.csv')]
ds = Dataset.Tabular.from_delimited_files(path=datastore_path)

ds = ds.take(3000).to_pandas_dataframe()

In [ ]:
compute_cluster_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
compute_cluster = ComputeTarget.create(ws, "compute-cluster", compute_cluster_config)

compute_cluster.wait_for_completion()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
#early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling( {
    '--fit_intercept': choice(1,0),
    '--normalize': choice(1,0)
})

if "training" not in os.listdir():
    os.mkdir("./training")
    
#TODO: Create your estimator and hyperdrive config
est = SKLearn(source_directory=".", compute_target=compute_cluster, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling, primary_metric_name="mean_squared_error", primary_metric_goal=PrimaryMetricGoal.MINIMIZE, max_total_runs=5, max_concurrent_runs=3, estimator=est )

In [ ]:
run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n RMSE:', best_run_metrics['RMSE'])
print('\n fit_intercept:', parameter_values[1])
print('\n normalize:', parameter_values[3])

In [ ]:
#Save the best model
best_run.get_file_names()
model = best_run.register_model(model_name='hyperdrivemodel', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
compute_cluster.delete()